# Ensemble Learning

## Initial Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

# Needed for decision tree visualization
from IPython.display import Image



Bad key "text.kerning_factor" on line 4 in
C:\Users\Scott Alexander\anaconda\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [3]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
df.head()


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [40]:
name_to_num = {name: num for num, name in enumerate(df.verification_status)if name}
df["verification_status"] = df["verification_status"].map(name_to_num)


## Split the Data into Training and Testing

In [41]:
print(name_to_num)

{68814: 68814, 68816: 68816, 68813: 68813}


In [11]:
#Create our features
# df=pd.DataFrame(np.random.randn(68817,86))            #training
# msk = np.random.rand(len(df)) < 0.8
# train = df[msk]
# test = df[~msk]

X = df.copy()
X.drop("loan_amnt", axis=1, inplace=True)
X=pd.get_dummies(X)

In [12]:

X.head()
y = df["verification_status"].ravel()
y[:5]


# # Create our targ

array([68814, 68816, 68816, 68816, 68813], dtype=int64)

In [13]:
df.columns

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

In [42]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [43]:
# Create the StandardScaler instance

scaler = StandardScaler()


In [44]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset


X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [45]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train, y_train)

In [46]:
# Calculated the balanced accuracy score
predictions = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)

In [47]:
# Display the confusion matrix
print("Confusion Matrix")
#display(cm_df)
print(f"Accuracy Score : {acc_score}")


Confusion Matrix
Accuracy Score : 0.4767800058122639


In [48]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

       68813       0.48      1.00      0.65      8203
       68814       0.00      0.00      0.00      6490
       68816       0.00      0.00      0.00      2512

    accuracy                           0.48     17205
   macro avg       0.16      0.33      0.22     17205
weighted avg       0.23      0.48      0.31     17205



C:\Users\Scott Alexander\anaconda\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Scott Alexander\anaconda\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Scott Alexander\anaconda\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
importances_df =pd.DataFrame(sorted(zip(importances, X.columns), reverse=True))

#importances_df.set_index(importances_df[1], inplace=True)
#importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importance'}, inplace=True)
#importances_sorted = importances_df.sort_values(by='Feature Importance')
importances_df.head(30)
#importances_sorted.plot(kind='barh', color='lightgreen',  title= 'FEATURE IMPORTANCE', legend=True)

,Feature Importance,1
0,0.647650,verification_status
1,0.013987,dti
2,0.010712,out_prncp
3,0.010674,annual_inc
4,0.010672,out_prncp_inv
5,0.009976,installment
6,0.009886,int_rate
7,0.009150,last_pymnt_amnt
8,0.009090,total_rec_int
9,0.008299,mo_sin_old_rev_tl_op


### Easy Ensemble Classifier

In [50]:
# Train the Classifier
X_scaler = scaler.fit(X_train)

In [51]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [52]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[8203,    0,    0],
       [   0, 6490,    0],
       [   0,    0, 2512]], dtype=int64)

In [53]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      68813       1.00      1.00      1.00      1.00      1.00      1.00      8203
      68814       1.00      1.00      1.00      1.00      1.00      1.00      6490
      68816       1.00      1.00      1.00      1.00      1.00      1.00      2512

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [ ]:
print("The Easy Ensemble Classifer had an accuracy score of 1, the previous model had an accuracy score of 47%")

In [ ]:
print("They all had a recall score of 1")

In [ ]:
print("they hall had a geo of 1")

In [ ]:
print("the top 3 features are verficiation_sdtatus, DTI, and out_prncp")